# Beam bending calculator
Reproduction of the example found [here](https://www.mathalino.com/reviewer/mechanics-and-strength-of-materials/solution-to-problem-448-relationship-between-load-shear).

Leave the uncommented script lines *as they are*; they ensure that all necessary modules are imported and active.

You can play around choosing different loads and application ranges.

In [ ]:
%matplotlib inline
from crunching import DistributedLoad, PointLoad, graphic_output

with graphic_output() as (beam, x):
    beam.length(9)           # Beam length
    beam.fixed_support(2)    # x-coordinate of the fixed support
    beam.rolling_support(7)  # x-coordinate of the rolling support

#     # SYNTAX A
#     # Add a point load (20kN pointing downward, at x=3m)
#     beam.point_load(-20, 3)
#     # Distributed load (10kN/m pointing downward, between x=3m and x=9m)
#     beam.distributed_load(-10, (3, 9))
#     # Distributed load (20kN/m pointing downward, between x=0m and x=2m)
#     beam.distributed_load(-20, (0, 2))
    
    # SYNTAX B
    beam.add_loads([PointLoad(-20, 3),
                    DistributedLoad(-20, (0, 2)),
                    DistributedLoad(-10, (3, 9))])

Click the link to download [a pdf file with this example](./output/test01.pdf "Come on and click! You know you want to.").